# MQTT

* [Web mqtt client](http://www.hivemq.com/demos/websocket-client/)
* Command line tools: mosquitto_pub, mosquitto_sub

In [4]:
%%host

from paho.mqtt.client import Client
from threading import Event
import time

USE_WS = False
BROKER = '10.39.40.200'

if USE_WS:
    PORT = 9001
    TRANSPORT = 'websockets'
else:
    PORT = 1883
    TRANSPORT = 'tcp'

# on_message runs on a different thread ...
connected = Event()

def on_message(client, userdata, message):
    print(f"message received: {message.topic} = {message.payload}")

def on_connect(client, userdata, flags, rc):
    global connected
    client.subscribe('#')
    connected.set()
    
def on_disconnect(client, userdata, rc):
    global connected
    connected.clear()
    print(f"disconnected, rc={rc}") 
    
client = Client("client-socks", transport=TRANSPORT)
client.on_message = on_message
client.on_connect = on_connect
client.on_disconnect = on_disconnect
client.connect(BROKER, PORT)

# start a separate thread listening to messages
client.loop_start()

# wait for connection before publishing
connected.wait()

for i in range(5):
    topic = f"demo/topic_{i}"
    msg   = f"msg {i}"
    print(f"\npublish {topic} = {msg}")
    client.publish(topic, msg)
    time.sleep(0.1)

# keep listening to messages
time.sleep(20)

client.disconnect()
print()


publish demo/topic_0 = msg 0
message received: demo/topic_0 = b'msg 0'

publish demo/topic_1 = msg 1
message received: demo/topic_1 = b'msg 1'

publish demo/topic_2 = msg 2
message received: demo/topic_2 = b'msg 2'

publish demo/topic_3 = msg 3
message received: demo/topic_3 = b'msg 3'

publish demo/topic_4 = msg 4
message received: demo/topic_4 = b'msg 4'

disconnected, rc=0
